In [6]:
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from graphviz import Source
from sklearn import tree
from IPython.display import SVG



## Project Settings

Specified here are the paths for the data and the features to run over in the list of best features.
Each entry in the list is a list containing one single string of the features to try, comma seperated. In this way it is easy to write a script to 
add entries to try very easily. 

In [7]:

##### set parameters
path_train_data = 'train.csv'
path_test_data = 'test.csv'
path_all_data = 'Dataset Correlated Removed.csv'

# set features here

best_features = s = [['nof_acetyl, nof_COOH, nof_NH2, nof_SH, nof_negCharge, nof_HBA, molLogP'],
     ['nof_COOH, negCharge/Volume, posCharge/Volume, nof_posCharge, nof_HBA, nof_Chirals, molLogS'],
     ['nof_COOH, nof_SO3H, nof_PO4, nof_HBA, nof_Chirals, PSA/Area, molLogP'],
     ['nof_COOH, negCharge/Volume, nof_negCharge, nof_HBD, PSA/Area, molArea, molLogP'],
     ['nof_COOH, nof_PO4, negCharge/Volume, C_R1, nof_negCharge, nof_Chirals, molLogP'],
     ['negCharge/Volume, nof_negCharge, nof_posCharge, nof_Chirals, PSA/Area, molArea, molLogP'],
     ['nof_COOH, posCharge/Volume, a_heavy, nof_negCharge, PSA/Area, molArea, molLogP'],
     ['nof_SO3H, negCharge/Volume, a_heavy, nof_negCharge, nof_Rings, nof_HBA, nof_Chirals'],
     ['nof_COOH, nof_SO3H, posCharge/Volume, nof_negCharge, nof_HBA, molArea, molLogP'],
     ['nof_COOH, nof_SH, nof_SO3H, nof_PO4, a_heavy, nof_HBA, molArea'],
     ['nof_COOH, nof_SO3H, nof_PO4, a_heavy, nof_Rings, nof_HBA, molArea'],
     ['nof_COOH, nof_SH, nof_SO3H, nof_PO4, nof_Rings, nof_HBA, molArea'],
     ['nof_COOH, nof_NH2, nof_PO4, negCharge/Volume, posCharge/Volume, nof_HBA, PSA/Area'],
     ['nof_COOH, nof_SH, a_heavy, nof_negCharge, nof_Rings, nof_HBA, molArea'],
     ['nof_COOH, nof_SH, nof_negCharge, nof_HBD, nof_HBA, molArea, molLogP'],
     ['nof_COOH, nof_SH, nof_SO3H, posCharge/Volume, a_heavy, nof_HBA, molArea'],
     ['nof_COOH, nof_SH, nof_SO3H, posCharge/Volume, nof_Rings, nof_HBA, molArea'],
     ['nof_COOH, nof_SH, nof_PO4, nof_negCharge, nof_Rings, nof_HBA, molArea'],
     ['nof_COOH, a_heavy, nof_negCharge, nof_HBD, nof_HBA, nof_Chirals, molArea'],
     ['nof_COOH, nof_SH, nof_SO3H, nof_negCharge, nof_HBA, molArea, molLogP'],
     ['nof_COOH, nof_PO4, posCharge/Volume, a_heavy, nof_Rings, nof_HBA, molArea'],
     ['nof_COOH, nof_SH, nof_SO3H, nof_negCharge, nof_Rings, nof_HBA, molArea'],
     ['nof_SO3H, a_heavy, nof_negCharge, nof_Rings, nof_HBA, nof_Chirals, molArea'],
     ['nof_COOH, nof_SO3H, nof_PO4, nof_HBD, nof_HBA, nof_Chirals, molLogP'],
     ['nof_COOH, nof_SO3H, nof_PO4, nof_negCharge, nof_HBA, molArea, molLogP'],
     ['nof_SH, nof_PO4, posCharge/Volume, nof_negCharge, nof_Rings, nof_HBA, molArea'],
     ['nof_COOH, posCharge/Volume, nof_negCharge, nof_Rings, nof_HBA, molArea, molLogP'],
     ['nof_COOH, nof_PO4, posCharge/Volume, nof_HBD, nof_HBA, nof_Chirals, molLogP'],
     ['nof_COOH, nof_SH, C_sp3, a_heavy, nof_negCharge, nof_posCharge, molLogP'],
     ['nof_COOH, nof_SH, nof_SO3H, C_sp3, a_heavy, nof_negCharge, nof_Chirals'],
     ['nof_SO3H, nof_PO4, C_sp3, a_heavy, nof_negCharge, nof_Chirals, molLogP'],
     ['nof_SH, nof_SO3H, C_sp3, a_heavy, nof_negCharge, nof_posCharge, nof_Chirals'],
     ['nof_acetyl, nof_SH, nof_SO3H, C_sp3, a_heavy, nof_negCharge, nof_Chirals'],
     ['nof_COOH, nof_NH2, nof_PO4, C_sp3, a_heavy, nof_negCharge, nof_Chirals'],
     ['nof_COOH, nof_SO3H, C_sp3, a_heavy, nof_negCharge, nof_posCharge, nof_Chirals'],
     ['PSA/Area, nof_SO3H, C_R0, nof_Rings, nof_OH, a_heavy, nof_Chirals'],
     ['nof_acetyl, nof_RotB, negCharge/Volume, C_R1, nof_HBA, nof_Chirals, molLogP']]



best_features = [list(map(str.strip, x[0].split(','))) for x in best_features]

k = len(best_features)


## Load Dataset

This code loads dataset into the variables below and converts the labels to categorical 0, 1 pairs.

In [8]:
# load dataset
all_data = pd.DataFrame(pd.read_csv(path_all_data))
all_labels = all_data['SLC'].astype('category').cat.codes
# drop labels
all_data.drop('SLC', axis=1, inplace=True)

train_data = pd.DataFrame(pd.read_csv(path_train_data))
train_labels = train_data['SLC'].astype('category').cat.codes
# drop labels

train_data.drop('SLC', axis=1, inplace=True)

test_data = pd.DataFrame(pd.read_csv(path_test_data))
test_labels = test_data['SLC'].astype('category').cat.codes
# drop labels
test_data.drop('SLC', axis=1, inplace=True)


## AUC and Classification Accuracy - Decision Tree

The code below will find the classification accuracy using 10-fold cross-validation using stratified sampling to help class imbalance. The AUC on the test split is also found.

In [12]:
# visualize decision tree for input features
''' HYPERPARAMS FOR DECISION TREE
 
 These parameters implement a rudimentary pruning algorithm, would ideally like to use AB pruning'''
enable_pruning = True
# maximum depth of dtree
max_depth = 5
# how many samples your need atleast, at a LEAF node
min_samples = 3

d_trees = []


# find CA - uses 10-fold cross validation 
# with stratified sampling to help with class imbalance
# and simple average over subsets
dt_cas = []


for i in range(k):
    aucs = []
    # make fold
    skf = StratifiedKFold(n_splits=10, shuffle=True)
    for trx, tex in skf.split(all_data, all_labels):
        # strip data to required features
        subset_data = all_data.filter(best_features[i], axis=1)
        
        # find auc
        dtree = DecisionTreeClassifier(presort=True, max_depth=max_depth, min_samples_leaf=min_samples)
        dtree.fit(subset_data.iloc[trx, :], all_labels.iloc[trx])        
        pred = dtree.predict(subset_data.iloc[tex, :])
        labels = all_labels.iloc[tex]
        
        acc = roc_auc_score(labels, pred)
        # record auc to average later
        aucs.append(acc)
    
    dt_cas.append(np.mean(aucs))
    

# find AUC 
dt_aucs = []
for i in range(k):
    subset_test_data = test_data.filter(best_features[i], axis=1)
    subset_train_data = train_data.filter(best_features[i], axis=1)
    
    clf = DecisionTreeClassifier(presort=True, max_depth=max_depth, min_samples_leaf=min_samples)
    clf.fit(subset_train_data, train_labels)
    d_trees.append(clf)
    
    # make its predictions on test data
    pred = d_trees[i].predict(subset_test_data)
    
    # find auc scores
    auc = roc_auc_score(test_labels, pred)
    
    # record the scores
    dt_aucs.append(auc)
    
print('Decision Tree Results:')
print('\tAUC\tAcc\tFeatures')
for i, f in enumerate(zip(dt_aucs, dt_cas)):
    print('\t%05.3f\t%05.3f\t' % tuple(f) + ', '.join(best_features[i]))

    

Decision Tree Results:
	AUC	Acc	Features
	0.690	0.695	nof_acetyl, nof_COOH, nof_NH2, nof_SH, nof_negCharge, nof_HBA, molLogP
	0.655	0.638	nof_COOH, negCharge/Volume, posCharge/Volume, nof_posCharge, nof_HBA, nof_Chirals, molLogS
	0.702	0.698	nof_COOH, nof_SO3H, nof_PO4, nof_HBA, nof_Chirals, PSA/Area, molLogP
	0.690	0.694	nof_COOH, negCharge/Volume, nof_negCharge, nof_HBD, PSA/Area, molArea, molLogP
	0.619	0.640	nof_COOH, nof_PO4, negCharge/Volume, C_R1, nof_negCharge, nof_Chirals, molLogP
	0.702	0.681	negCharge/Volume, nof_negCharge, nof_posCharge, nof_Chirals, PSA/Area, molArea, molLogP
	0.714	0.656	nof_COOH, posCharge/Volume, a_heavy, nof_negCharge, PSA/Area, molArea, molLogP
	0.690	0.672	nof_SO3H, negCharge/Volume, a_heavy, nof_negCharge, nof_Rings, nof_HBA, nof_Chirals
	0.774	0.636	nof_COOH, nof_SO3H, posCharge/Volume, nof_negCharge, nof_HBA, molArea, molLogP
	0.536	0.534	nof_COOH, nof_SH, nof_SO3H, nof_PO4, a_heavy, nof_HBA, molArea
	0.702	0.713	nof_COOH, nof_SO3H, nof_PO4, a_hea

## AUC and Classification Accuracy - Random Forest Walk

The code below will find the classification accuracy using 10-fold cross-validation using stratified sampling to help class imbalance. The AUC on the test split is also found.

In [10]:
# visualize random forest features
rfws = []

# find CA - uses 10-fold cross validation 
# with stratified sampling to help with class imbalance
# and simple average over subsets
rfw_cas = []

for i in range(k):
    aucs = []
    # make fold
    skf = StratifiedKFold(n_splits=10, shuffle=True)
    for trx, tex in skf.split(all_data, all_labels):
        # strip data to required features
        subset_data = all_data.filter(best_features[i], axis=1)
        
        # find auc
        rfwtree = RandomForestClassifier(n_estimators=100)
        rfwtree.fit(subset_data.iloc[trx, :], all_labels.iloc[trx])
        pred = rfwtree.predict(subset_data.iloc[tex, :])
        labels = all_labels.iloc[tex]
        
        acc = roc_auc_score(labels, pred)
        # record auc to average later
        aucs.append(acc)
    
    rfw_cas.append(np.mean(aucs))
    

# find AUC 
rfw_aucs = []
for i in range(k):
    subset_test_data = test_data.filter(best_features[i], axis=1)
    subset_train_data = train_data.filter(best_features[i], axis=1)
    
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(subset_train_data, train_labels)
    rfws.append(clf)
    
    # make its predictions on test data
    pred = rfws[i].predict(subset_test_data)
    
    # find auc scores
    auc = roc_auc_score(test_labels, pred)
    
    # record the scores
    rfw_aucs.append(auc)

print('Random Forest Results:')
print('\tAUC\tAcc\tFeatures')
for i, f in enumerate(zip(rfw_aucs, rfw_cas)):
    print('\t%05.3f\t%05.3f\t' % tuple(f) + ', '.join(best_features[i]))


Random Forest Results:
	AUC	Acc	Features
	0.690	0.685	nof_acetyl, nof_COOH, nof_NH2, nof_SH, nof_negCharge, nof_HBA, molLogP
	0.750	0.710	nof_COOH, negCharge/Volume, posCharge/Volume, nof_posCharge, nof_HBA, nof_Chirals, molLogS
	0.857	0.728	nof_COOH, nof_SO3H, nof_PO4, nof_HBA, nof_Chirals, PSA/Area, molLogP
	0.810	0.683	nof_COOH, negCharge/Volume, nof_negCharge, nof_HBD, PSA/Area, molArea, molLogP
	0.738	0.626	nof_COOH, nof_PO4, negCharge/Volume, C_R1, nof_negCharge, nof_Chirals, molLogP
	0.845	0.707	negCharge/Volume, nof_negCharge, nof_posCharge, nof_Chirals, PSA/Area, molArea, molLogP
	0.833	0.699	nof_COOH, posCharge/Volume, a_heavy, nof_negCharge, PSA/Area, molArea, molLogP
	0.762	0.711	nof_SO3H, negCharge/Volume, a_heavy, nof_negCharge, nof_Rings, nof_HBA, nof_Chirals
	0.786	0.663	nof_COOH, nof_SO3H, posCharge/Volume, nof_negCharge, nof_HBA, molArea, molLogP
	0.583	0.583	nof_COOH, nof_SH, nof_SO3H, nof_PO4, a_heavy, nof_HBA, molArea
	0.690	0.706	nof_COOH, nof_SO3H, nof_PO4, a_hea

## Visualizing individual decision trees

The tree in variable `dtree` is visualized by the cell below. We can see how it is pruned, the splitting rule, etc.

In [11]:
i = 0

dtree = d_trees[8]
graph = Source(tree.export_graphviz(dtree, out_file=None, feature_names=best_features[i][:dtree.n_features_]))
SVG(graph.pipe(format='svg'))            
graph = Source( tree.export_graphviz(dtree, out_file=None, feature_names=best_features[i][:dtree.n_features_]))
graph.format = 'png'
graph.render('dtree_render',view=True)
graph = Source( tree.export_graphviz(dtree, out_file=None, feature_names=best_features[i][:dtree.n_features_]))
png_bytes = graph.pipe(format='png')
with open('dtree_pipe.png','wb') as f:
    f.write(png_bytes)
Image(png_bytes)

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH

## Feature importance

The feature importances are compared below for decision trees and random forests.
Reported below is code to visualize all decision trees. This requires the graphviz package and has some bugs, which will be reported. This code visualizes all decision trees and finds the feature importances for all of them.

In [ ]:
i = 0
# visualization
for dtree in d_trees:
    if i < k:
        print('Feature importances for tree and forest (resp.) %s/%s:' % (i + 1, k))
        for e in zip(dtree.feature_importances_, rfws[i].feature_importances_, best_features[i]):
            print('\t%6f\t%6f\t%s' % e)
        
        try:
            graph = Source(tree.export_graphviz(dtree, out_file=None, feature_names=best_features[i][:dtree.n_features_]))
            SVG(graph.pipe(format='svg'))            
            graph = Source( tree.export_graphviz(dtree, out_file=None, feature_names=best_features[i][:dtree.n_features_]))
            graph.format = 'png'
            graph.render('dtree_render',view=True)
            graph = Source( tree.export_graphviz(dtree, out_file=None, feature_names=best_features[i][:dtree.n_features_]))
            png_bytes = graph.pipe(format='png')
            with open('dtree_pipe.png','wb') as f:
                f.write(png_bytes)
            Image(png_bytes)
        except:
            print('Something went wrong with rendering graph')
    else:
        print('Warning, code may be buggy')
    i += 1